In [30]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import threading
from dash import Dash, dcc, html, Input, Output, jupyter_dash

%matplotlib inline

In [31]:
mapbox_token = "pk.eyJ1IjoidmVydWJlZSIsImEiOiJjbTJxOXR5ZngxMTF1Mmtxenp6aW16eTdjIn0.N-u6aPAAJvjY44ZLWfZPmw"

px.set_mapbox_access_token(mapbox_token)

In [32]:
df = pd.read_csv('Final_projects_files/project_1_python.csv', index_col='index',sep=',', decimal='.', encoding='UTF-8')

df.tail(1)

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,hosp_patients,total_tests,...,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,population,median_age,gdp_per_capita,life_expectancy,latitude,longitude
index,,,,,,,,,,,,,,,,,,,,,
190612,ZWE,Africa,Zimbabwe,2022-06-18,254753.0,NaN,5533.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,15092171.0,19.6,1899.775,61.49,-20.0,30.0


In [33]:
df['max_cases'] = df.groupby(['location', 'population', 'continent', 'latitude', 'longitude'])['total_cases'].transform('max')

df['max_deaths'] = df.groupby(['location', 'population', 'continent', 'latitude', 'longitude'])['total_deaths'].transform('max')

df['max_tests'] = df.groupby(['location', 'population', 'continent', 'latitude', 'longitude'])['total_tests'].transform('max')

df['max_vax'] = df.groupby(['location', 'population', 'continent', 'latitude', 'longitude'])['total_vaccinations'].transform('max')

df['max_people_vax'] = df.groupby(['location', 'population', 'continent', 'latitude', 'longitude'])['people_fully_vaccinated'].transform('max')

In [34]:


metric_dict = {'max_cases': 'Total cases', 
               'max_deaths': 'Total deaths', 
               'max_tests': 'Total tests', 
               'max_vax': 'Total vaccinations', 
               'max_people_vax': 'Number of fully vaccinated people'}

continent_list = list(df['continent'].drop_duplicates())

second_dash_app = Dash(__name__)

second_dash_app.layout = html.Div([
    html.Div([
        html.H1('COVID-19 Tracker', style={"color": "#3498DB", "text-align": "center"})
    ]),                           
    html.H3("Choose continent:", style={"text-align": "center", "color": "#AED6F1"}),
    dcc.Dropdown(
        id='continent', 
        options=[{'value': str(continent), 'label': continent} 
                 for continent in continent_list],
        value="Europe",
        style= {"text-align": "center"}
        ),
    html.H3("Choose metric:", style={"text-align": "center", "color": "#AED6F1"}),
    dcc.Dropdown(
        id='metric', 
        options=[{'label': metric.replace('_', ' ').title(), 'value': metric} for metric in metric_dict],
        value='max_cases',
        style= {"text-align": "center"}
        ),       
    html.Br(),
    html.Div(children=[
        dcc.Graph(id="map"),
    ])

])

@second_dash_app.callback(
    Output(component_id='map', component_property='figure'),
    [Input(component_id='continent', component_property='value'),
     Input(component_id='metric', component_property='value')]
)
def generate_covid_map(continent, metric):
    
    filtered_df = df[df['continent'] == continent]

   
    filtered_df[metric] = filtered_df[metric].fillna(0)
    
    map = px.scatter_mapbox(
        filtered_df,
        lat="latitude",
        lon="longitude",
        size=filtered_df[metric],
        hover_name="location",
        hover_data={'location': True},
        mapbox_style='dark',
        zoom=1,
        height=600,
        width=1040,
        size_max=15,
        title=f'COVID-19 - {metric_dict[metric]} in {continent}'
    )
    
    return map

if __name__ == '__main__':
    second_dash_app.run_server(mode='inline', port=8055)



/var/folders/ly/b4ncvmb902ggx_hbc9vc3qy40000gp/T/ipykernel_39329/3839566026.py:47: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/ly/b4ncvmb902ggx_hbc9vc3qy40000gp/T/ipykernel_39329/3839566026.py:47: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/ly/b4ncvmb902ggx_hbc9vc3qy40000gp/T/ipykernel_39329/3839566026.py:47: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th